In [40]:
import pandas as pd
import requests
import seaborn as sb

Matplotlib is building the font cache; this may take a moment.


In [62]:
teststarttime = '2025-04-05T00:00:46'
testendtime = '2025-05-05T17:05:58'
query = f'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={teststarttime}&endtime={testendtime}'
data = requests.get(query)

In [ ]:
data.json()

In [ ]:
df2 = pd.json_normalize(data.json(),"features")
df2.head()

In [3]:
def extract_api(starttime,endttime):
    try:
        query = f'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={starttime}&endtime={endttime}'
        data = requests.get(query)
        df = pd.json_normalize(data.json(),'features')
        return df

    except Exception as e:
        print(f'unable to retrieve data: {e} ')
        

In [ ]:
teststarttime = '2025-04-05T00:00:46'
testendtime = '2025-05-05T17:05:58'
#'2025-05-05T12:00:00'
#'2025-05-06T12:00:00'
dfdirty = extract_api(teststarttime,testendtime)
dfdirty.head()


In [102]:
df = dfdirty.copy()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df.columns

In [103]:
columns = [

    'properties.updated','properties.tz','properties.url','properties.detail',
    'properties.felt','properties.cdi','properties.mmi','properties.alert',
    'properties.status','properties.tsunami','properties.sig','properties.net',
    'properties.code','properties.ids','properties.sources','properties.types',
    'properties.nst','properties.dmin','properties.rms','properties.gap',
    'properties.magType','properties.title','geometry.type', 'type'  
]
df = df.drop(columns=columns)
df = df.dropna(subset=['properties.mag'])

In [104]:
names = {
    'properties.type':'type',
    'properties.mag':'magnitude',
    'properties.place':'location',
    'properties.time':'time',
}
df = df.rename(columns = names)

In [ ]:
df['properties.time'] = pd.to_datetime(df['properties.time'],unit='ms')

In [106]:
df_coordinates = df['geometry.coordinates'].apply(pd.Series).round(2)
df_coordinates.columns = ['longitude','latitude','depth']

df.drop(columns=['geometry.coordinates'], inplace=True)
df = pd.merge(df,df_coordinates,left_index=True,right_index=True)

In [ ]:
df

In [107]:
def nearest_city(location):
    if ',' in location:
        return location.split(',')[0]
    else:
        return 'None'

df['nearestCity'] = df['location'].apply(nearest_city)
df['nearestCity'].isnull().sum()

np.int64(0)

In [108]:
df['location'] = df['location'].apply(lambda x: x.split(',')[-1].strip())

In [ ]:
df['time']

In [ ]:
df['time'].dt.floor('S')

In [ ]:
df['magnitude'].transform(lambda x:(x/x.mean()*100))

In [109]:
len(df['location'][0])

2

In [ ]:

df2 = df[df['location'].str.len() == 2]

df2

In [ ]:
df2['location'].unique()